## Chapter4. 신경망 학습

### 4.1. 데이터 기반학습
* 신경망은 입력(x)에 대해 올바른 출력(y)을 결정하는 가중치(w)를 정하는 것이 관건이다.
* 2개층을 갖는 XOR같은 간단한 신경망의 경우 가중치가 몇 개 안되지만 풀려고 하는 문제에 따라 가중치의 개수는 늘어난다. 수천개에서 수만개가 넘을 수 도 있다. 그 가중치들을 사람이 계산해서 입력하는 것은 불가능에 가깝다.
* 그렇다면 그 가중치를 계산하는 것이 관건인데 신경망은 역전파 알고리즘을 통해 학습한다.
* 여기서는 신경망 학습에 대한 여러 요소들을 익히고 다음장에서 역전파 알고리즘을 구현하자.

### 4.2. 손실함수
* 신경망의 출력과 실제 값의 차이에 대한 함수이다.
* 손실함수는 loss function 또는 cost function 이라고 한다.

#### 4.2.1 평균 제곱 오차(mean squared error, MES)
![](https://latex.codecogs.com/gif.latex?E%3D%5Cfrac%20%7B%201%20%7D%7B%202%20%7D%20%5Csum%20_%7B%20K%20%7D%5E%7B%20%7D%7B%20%7B%20%28%7B%20y%20%7D_%7B%20k%20%7D-%7B%20t%20%7D_%7B%20k%20%7D%29%20%7D%5E%7B%202%20%7D%20%7D)
* 출력y와 정답t의 차이의 제곱의 합에 1/2한 결과입니다.
* 구현체는 다음과 같습니다.

In [6]:
import numpy as np
def mean_squared_error(y,t):
    return 0.5 * np.sum((y-t)**2)

y = [0.05, 0.05, 0.6, 0.3] #출력
t = [0, 0, 1, 0] #실제 정답은 3번째 원소에 해당

mean_squared_error(np.array(y),np.array(t))

0.1275

#### 4.2.2 교차 엔트로피 오차(cross entropy error, CEE)
![](https://latex.codecogs.com/gif.latex?E%3D-%5Csum%20_%7B%20K%20%7D%5E%7B%20%7D%7B%20%7B%20t%20%7D_%7B%20k%20%7D%5Clog%20%7B%20%7B%20y%20%7D_%7B%20k%20%7D%20%7D%20%7D)
* log는 밑이 e인 자연로그이다. 
* 정답이 true/false인 경우 t가 1또는 0 이기때문에 교차 엔트로피의 결과는 y의 자연로그이거나 0이다.
* 다음은 자연 로그에 대한 그래프이다.
![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Log.svg/300px-Log.svg.png)
* log함수의 x가 1일때는 0, x가 0에 가까워 질수록 음수로 떨어진다. 
* 즉 신경망 출력y의 값이 크고 정답이면 0, 정답이 아니면 작은수를 합쳐 부호를 변환하여 오차를 구한다.
> 신경망의 출력백터의 요소가 (0~1) 범위의 스칼라 조합일 경우를 생각해보면 정답레이블(t=1)일때 y가 1미만일경우, 음수대의 숫자를 자연로그는 반환하기 때문에, 다 더해서 부호반전을 통해 에러를 계산한다.

In [8]:
def cross_entropy_error(y,t):
    delta = 1e-7
    return -1*np.sum(t*np.log(y+delta))

cross_entropy_error(np.array(y),np.array(t))

0.51082545709933802

#### 4.2.3 미니배치 학습(mini-batch)
* 실제 학습데이터를 기반으로 손실함수를 구할 때, 데이터가 100개면 100개의 손실함수를 계산해야한다.
* 아래는 n개의 데이터에 대한 교차 엔트로피 오차 공식이다.
![](https://latex.codecogs.com/gif.latex?E%3D-%5Cfrac%20%7B%201%20%7D%7B%20N%20%7D%20%5Csum%20_%7B%20n%20%7D%5E%7B%20%7D%7B%20%5Csum%20_%7B%20k%20%7D%5E%7B%20%7D%7B%20%7B%20t%20%7D_%7B%20nk%20%7Dlog%7B%20y%20%7D_%7B%20nk%20%7D%20%7D%20%7D)
* 데이터가 무수히 많아질 때는 학습 시간등을 고려하여 모든 데이터 중 일부를 무작위로 선택해 학습을 진행한다.
* 이것이 미니배치이다.
* 아래는 교차 엔트로피 오차에 대한 미치매치용 실행 함수이다(t가 원핫인코딩일경우).

In [13]:
def cross_entropy_error(y,t):
    if y.dim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size),t]))

#### 4.2.4 손실함수의 역활
* 신경망에서 정확한 가중치(weight)를 구하기 위해서 '정확도'가 아닌 '오차률'(손실함수)을 사용한다.
* 이유는 신경망 학습(가중치 학습)에서 매개변수의 미분을 계산해서 이동방향을 결정하기 때문이다.
* 즉 손실함수의 미분값(기울기)의 음,양을 판단해 매개변수를 갱신한다. 미분관점에서는 매개변수의 값을 조금 변화시켰을때 손실함수의 결과 값이 이동할 방향을 결정하기 때문이다.
* 미분값이 0이면 어느 쪽으로 움직여도 손실 함수의 값이 달라지지 않기 때문에 학습이 멈춘다.
* 만약 정확도를 지표로 삼으면 매개변수의 미분이 대부분의 장소에서 0이 된다. 정확도는 매개변수의 변화에 따라 32%에서 32.0123%과 같은 연속적 변화보다는 33%, 34%와 같은 불연속적인 값으로 튀기 때문에 학습에 대한 최적화가 어렵다.
* 이것은 계단함수를 쓰기 않는 이유와 동일한다.
* 그래서 손실함수의 미분을 이용하여 학습한다.
* **여기서 핵심포인트는 기울기가 0이 되지 않는 덕분에 신경망이 올바르게 학습할 수 있는것이다.**

### 4.3. 수치 미분
#### 4.3.1 미분
![](https://latex.codecogs.com/gif.latex?%5Cfrac%20%7B%20df%28x%29%20%7D%7B%20dx%20%7D%20%3D%5Clim%20_%7B%20h-%3E0%20%7D%7B%20%5Cfrac%20%7B%20f%28x&plus;h%29-f%28x%29%20%7D%7B%20h%20%7D%20%7D)
* 오차
#### 4.3.2 편미분
* 여러 변수가 존재하는 함수에서 각 변수에 대한 미분
*

In [14]:
def numerical_diff(f,x):
    h = 1e-4
    return (f(x+h)-f(x-h)) / (2*h)

### 4.4 기울기
* 아래 처럼 모든 변수에대한 편미분을 백터로 정리한것을 기울기(gradient)라고 한다.
![](https://latex.codecogs.com/gif.latex?%28%5Cfrac%20%7B%20%5Cpartial%20f%20%7D%7B%20%5Cpartial%20%7B%20x%20%7D_%7B%200%20%7D%20%7D%20%2C%5Cfrac%20%7B%20%5Cpartial%20f%20%7D%7B%20%5Cpartial%20%7B%20x%20%7D_%7B%201%20%7D%20%7D%20%29)


#### 4.4.1 경사 하강법
* 신경망 학습시 복잡한 매개변수 공간에서 손실함수를 최소화하는 매개변수를 구해야한다.
* 이런 상황에서 기울기를 잘 활용해 최솟값을 찾는 것이 경사법(gradient)이다.
* 다음은 경사법의 수식이다.
![](https://latex.codecogs.com/gif.latex?%7B%20x%20%7D_%7B%200%20%7D%3D%7B%20x%20%7D_%7B%200%20%7D-%5Ceta%20%5Cfrac%20%7B%20%5Cpartial%20f%20%7D%7B%20%5Cpartial%20%7B%20x%20%7D_%7B%200%20%7D%20%7D%20%5C%5C%20%7B%20x%20%7D_%7B%201%20%7D%3D%7B%20x%20%7D_%7B%201%20%7D-%5Ceta%20%5Cfrac%20%7B%20%5Cpartial%20f%20%7D%7B%20%5Cpartial%20%7B%20x%20%7D_%7B%201%20%7D%20%7D)
* 곱해지는 상수는 학습률이다.
* 위 수식이 반복되면서 값을 갱신(학습)한다.

* 최소값 최대값 여부에 따라 gradient descent, gradient ascent로 구별되지만 최소,최대를 구별하는것은 본질적으로 중요하지 않다. 보통 머신러닝에서는 최소값을 찾는 gradient descent를 많이 사용한다.
* 

#### 4.4.2 신경망에서 기울기
*
*
*

In [15]:
def numerical_gradient(f,x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(2)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
        
    return grad


def gredient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f,x)
        x -= lr * grad
    return

### 4.5 몇 가지 용어들
* 학습률 값이 사용자가 직접 정의해야 하는 파라미터를 하이퍼파라미터라(hyper parameter)한다.
* 모든 데이터(미니 배치는 선택된 데이터)를 한번 학습한것을 1에폭(epoch)이라 한다.